In [2]:
from qconv import TBNConv2d
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np


/opt/conda/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 1. Test SubArray and ADC Module

In [2]:
i = 0
j = 0
inputt = torch.randn(1, 64, 4, 4)
weightb = torch.randn(4, 64, 3, 3)
mask = torch.zeros_like(weightb)
inputT = torch.sign(inputt) # ternary bit
weightB = torch.sign(weightb) # binary weight bit
#print(inputT)
#print(weightB)
mask[:, :, i, j] = 1
weightB = weightB * mask
out_int = F.conv2d(inputT, weightB, None, 1, 1)
print(out_int)

tensor([[[[  0.,   0.,   0.,   0.],
          [  0.,  -6.,   6., -12.],
          [  0.,  -4.,  12.,   2.],
          [  0.,  -2.,  -4.,   6.]],

         [[  0.,   0.,   0.,   0.],
          [  0., -10.,   2.,   8.],
          [  0.,  -4.,   8.,   6.],
          [  0.,  10.,   0.,  -6.]],

         [[  0.,   0.,   0.,   0.],
          [  0.,   6.,  -2.,   4.],
          [  0.,   0.,   4., -10.],
          [  0.,  14.,   0.,  14.]],

         [[  0.,   0.,   0.,   0.],
          [  0.,   4.,  -8.,   6.],
          [  0.,   2.,  -2.,   4.],
          [  0., -12.,  -6.,   0.]]]])


In [19]:
print("input sign :\n",inputT)
print("weight sign : \n", weightB)
print(inputT[:,:,0,0])
print(weightB[:,:,0,0])
print((inputT[:,:,0,0] * weightB[0,:,0,0]).sum())

input sign :
 tensor([[[[ 1., -1.,  1., -1.],
          [-1., -1., -1.,  1.],
          [-1.,  1., -1., -1.],
          [ 1., -1., -1.,  1.]],

         [[ 1., -1.,  1., -1.],
          [-1.,  1., -1.,  1.],
          [-1., -1.,  1., -1.],
          [ 1., -1.,  1.,  1.]],

         [[ 1., -1.,  1.,  1.],
          [ 1., -1.,  1.,  1.],
          [-1.,  1.,  1., -1.],
          [ 1.,  1.,  1., -1.]],

         ...,

         [[-1., -1., -1.,  1.],
          [-1.,  1.,  1., -1.],
          [-1., -1., -1., -1.],
          [ 1.,  1.,  1., -1.]],

         [[-1., -1.,  1.,  1.],
          [ 1.,  1.,  1.,  1.],
          [ 1.,  1., -1.,  1.],
          [ 1., -1.,  1.,  1.]],

         [[-1.,  1.,  1.,  1.],
          [ 1.,  1., -1.,  1.],
          [-1., -1.,  1.,  1.],
          [-1., -1.,  1.,  1.]]]])
weight sign : 
 tensor([[[[-1., -0.,  0.],
          [-0., -0.,  0.],
          [ 0.,  0., -0.]],

         [[-1., -0., -0.],
          [-0.,  0.,  0.],
          [-0., -0., -0.]],

        

In [21]:
i = 0
j = 0
inputt = torch.randn(1, 64, 4, 4)
weightb = torch.randn(4, 64, 3, 3)
inputT = torch.sign(inputt) # ternary bit
weightB = torch.sign(weightb) # binary weight bit
out_int = F.conv2d(inputT, weightB, None, 1, 1)
out_check = torch.zeros_like(out_int)

for i in range(weightB.shape[2]):
    for j in range(weightB.shape[3]):
        mask = torch.zeros_like(weightb)
        #print(inputT)
        #print(weightB)
        mask[:, :, i, j] = 1
        weight_mask = weightB * mask
        out_partial = F.conv2d(inputT, weight_mask, None, 1, 1)
        out_check += out_partial
print((out_int - out_check).sum())

tensor(0.)


In [8]:
def adc_quantize(inputs, sub_array, adc_mode, adc_bits):
    """
    inputs: input tensor
    sub_array: number of sub-array
    adc_bits: number of adc bits
    """
    if adc_mode =='linear':
        min_val = -sub_array
        max_val = sub_array
        step_size = (max_val - min_val) / (2 ** adc_bits)
        index = torch.clamp(torch.floor((inputs-min_val) / step_size), 0, 2**adc_bits-1) # 0 ~ 2**adc_bits -1 mapping
        y = index * step_size + min_val
    elif adc_mode == 'none':
        y = inputs
        index =None
        step_size = None
    elif adc_mode =='original':
        min_val = inputs.min()
        max_val = inputs.max()
        step_size = (max_val - min_val).item() / (2 ** adc_bits)
        index = torch.clamp(torch.floor((inputs-min_val) / step_size), 0, 2**adc_bits-1) # 0 ~ 2**adc_bits -1 mapping
        y = index * step_size + min_val.item()
    else:
        raise NotImplementedError
    return step_size, index, y

In [12]:
print("original int : ", out_int)
step_size, index, y = adc_quantize(out_int, 128, 'linear', 4)
print("subarray linear")
print("step_size : ", step_size)
print("index : ", index)
print("y : ", y)
print("sign check :", (out_int +1e-6).sign() == (y+1e-6).sign())
step_size, index, y = adc_quantize(out_int, 128, 'original', 6)
print("neurosim ADC")
print("step_size : ", step_size)
print("index : ", index)
print("y : ", y)


original int :  tensor([[[[  0.,   0.,   0.,   0.],
          [  0.,  -6.,   6., -12.],
          [  0.,  -4.,  12.,   2.],
          [  0.,  -2.,  -4.,   6.]],

         [[  0.,   0.,   0.,   0.],
          [  0., -10.,   2.,   8.],
          [  0.,  -4.,   8.,   6.],
          [  0.,  10.,   0.,  -6.]],

         [[  0.,   0.,   0.,   0.],
          [  0.,   6.,  -2.,   4.],
          [  0.,   0.,   4., -10.],
          [  0.,  14.,   0.,  14.]],

         [[  0.,   0.,   0.,   0.],
          [  0.,   4.,  -8.,   6.],
          [  0.,   2.,  -2.,   4.],
          [  0., -12.,  -6.,   0.]]]])
subarray linear
step_size :  16.0
index :  tensor([[[[8., 8., 8., 8.],
          [8., 7., 8., 7.],
          [8., 7., 8., 8.],
          [8., 7., 7., 8.]],

         [[8., 8., 8., 8.],
          [8., 7., 8., 8.],
          [8., 7., 8., 8.],
          [8., 8., 8., 7.]],

         [[8., 8., 8., 8.],
          [8., 8., 7., 8.],
          [8., 8., 8., 7.],
          [8., 8., 8., 8.]],

         [[8.,

## Test FP Conv and Simulated Sign Conv

Ternary input * binary weight = Sa * Sw * Sign(ternary input) * Sign (binary weight)

In [30]:
k = 100
## 1. generate ternary input
inputs = torch.randn(3, k)
inputs_scale = torch.max(inputs)/2
inputs_sign = torch.zeros_like(inputs)
inputs_sign[inputs > inputs_scale] = 1.
inputs_sign[inputs < -inputs_scale] = -1.
print("Ternary input generate")
print("original input : \n", inputs)
print("inputs scale : ",inputs_scale)
print("input sign :", inputs_sign)
weights = torch.randn(k,4)
weights_scale = torch.max(weights)/2
weights_sign = weights.sign()
print("Binary weight generate")
print("original weight : \n", weights)
print("weights scale : ",weights_scale)
print("weights sign :", weights_sign)

print("First Matmul Sa*Sw*input_sign*weight_sign")
sasw_output= inputs_scale*weights_scale*torch.matmul(inputs_sign, weights_sign)
ori_output = torch.matmul(inputs_sign*inputs_scale, weights_sign*weights_scale)
print("sasw_output : \n", sasw_output)
print("ori_output : \n", ori_output)
print("sasw_output - ori_output : \n", sasw_output - ori_output)

Ternary input generate
original input : 
 tensor([[ 3.7617e-01, -3.2590e-02, -2.5216e-02, -3.0268e-01, -4.4702e-01,
         -1.5053e+00,  2.4570e-01,  1.0646e+00,  4.7700e-01, -1.4619e+00,
         -1.0889e+00,  2.3967e+00, -6.7485e-01,  1.3291e+00,  2.6701e-01,
          3.0953e-01,  6.9812e-01,  9.6879e-01,  2.6779e-01, -7.5830e-02,
          8.9461e-01,  5.6038e-02, -2.4166e-01,  5.0349e-01,  4.6164e-01,
          1.8369e+00, -5.0917e-01, -7.8674e-01,  1.5061e+00,  3.2744e-01,
         -5.6702e-04, -4.0379e-01, -2.9398e-01,  8.0342e-01,  1.1063e-01,
          3.7257e-02, -1.8955e-01,  1.1329e+00, -2.4125e+00, -2.1196e+00,
         -2.0303e+00, -9.7866e-01,  1.0811e+00,  3.0124e-01, -8.3483e-01,
         -1.9617e-01,  7.4695e-02, -3.8308e-01, -1.5923e-01, -9.8950e-01,
          1.4363e+00, -1.2340e+00,  6.1817e-01, -6.7230e-01,  2.2661e+00,
         -1.4797e-01,  1.7683e-02,  3.2814e-01, -4.1115e-01,  7.5125e-01,
          3.3428e-01, -2.1550e-01, -2.6335e-02, -6.7241e-01,  1.3156e+

## Test TBNConv
### 2.0 Test CPU mode

In [13]:

tbnconv = TBNConv2d(64, 2, 3, 1, 1, adc_mode=True)
inputs = torch.randn(1, 64, 4, 4)
tbnconv.pim_mode=True
tbnconv.adc_mode='linear'
tbnconv.adc_bits=5
linear_adc_out = tbnconv(inputs)
tbnconv.adc_mode='original'
original_out = tbnconv(inputs)
tbnconv.adc_mode='none'
out = tbnconv(inputs)
print("modified ADC out \n",linear_adc_out)
print("original ADC out \n", original_out)
print("Without ADC out \n", out)
print("modified adc diff : ", linear_adc_out - out)
print("original adc diff : ", original_out - out)
print("diff mse : ", F.mse_loss(linear_adc_out, out))
print("diff original_adc mse : ", F.mse_loss(original_out, out))


modified ADC out 
 tensor([[[[-0.1346, -0.4037, -0.5382,  0.1346],
          [-0.6728, -0.2691, -0.6728, -0.6728],
          [ 0.0000, -0.4037, -1.3456, -0.2691],
          [-0.2691, -0.6728,  0.2691, -0.4037]],

         [[-0.4037, -0.1346, -0.6728,  0.0000],
          [ 0.1346, -0.4037, -0.4037, -0.5382],
          [-0.4037, -0.6728,  0.1346, -0.4037],
          [-0.4037, -0.4037, -0.2691,  0.2691]]]], grad_fn=<AddBackward0>)
original ADC out 
 tensor([[[[ 0.0168,  0.0394, -0.2113,  0.2392],
          [-0.3232,  0.1340, -0.1782, -0.2392],
          [ 0.1750,  0.2901, -0.5524,  0.0184],
          [-0.1508, -0.3511,  0.4100, -0.2554]],

         [[-0.1325,  0.2160, -0.2686,  0.1561],
          [ 0.2964,  0.2181,  0.1761, -0.3774],
          [-0.1225, -0.1135,  0.5897, -0.1682],
          [-0.2512,  0.0284,  0.0315,  0.4736]]]], grad_fn=<AddBackward0>)
Without ADC out 
 tensor([[[[ 0.0673,  0.0841, -0.1682,  0.2859],
          [-0.2523,  0.1850, -0.1346, -0.1850],
          [ 0.2355,  0

### 2.1 Test GPU mode

In [5]:

tbnconv = TBNConv2d(64, 2, 3, 1, 1, adc_mode=True).cuda()
inputs = torch.randn(1, 64, 4, 4).cuda()
tbnconv.pim_mode=True
tbnconv.adc_mode='linear'
tbnconv.adc_bits=5
linear_adc_out = tbnconv(inputs)
tbnconv.adc_mode='original'
original_out = tbnconv(inputs)
tbnconv.adc_mode='none'
out = tbnconv(inputs)
print("modified ADC out \n",linear_adc_out)
print("original ADC out \n", original_out)
print("Without ADC out \n", out)
print("modified adc diff : ", linear_adc_out - out)
print("original adc diff : ", original_out - out)
print("diff mse : ", F.mse_loss(linear_adc_out, out))
print("diff original_adc mse : ", F.mse_loss(original_out, out))


modified ADC out 
 tensor([[[[ 0.0000, -0.2684,  0.0000, -0.2684],
          [-0.4026, -0.4026, -0.2684, -0.1342],
          [-0.5368, -0.2684,  0.0000, -0.5368],
          [-0.1342, -0.4026, -0.1342, -0.2684]],

         [[-0.5368, -0.4026, -0.6709,  0.0000],
          [ 0.0000, -0.9393, -0.1342, -0.5368],
          [-0.6709, -0.8051, -0.1342, -0.5368],
          [-0.1342, -0.5368, -0.1342,  0.4026]]]], device='cuda:0',
       grad_fn=<AddBackward0>)
original ADC out 
 tensor([[[[ 0.0734,  0.1394,  0.2023, -0.0262],
          [-0.1258,  0.0713,  0.2254,  0.1693],
          [-0.1599,  0.1730,  0.5415, -0.2359],
          [-0.0121, -0.0744,  0.0912, -0.0514]],

         [[-0.2930, -0.0367, -0.2589,  0.1090],
          [ 0.3507, -0.2825,  0.4450, -0.3056],
          [-0.3999, -0.4953,  0.1704, -0.1394],
          [ 0.0341, -0.2637, -0.0278,  0.4817]]]], device='cuda:0',
       grad_fn=<AddBackward0>)
Without ADC out 
 tensor([[[[ 1.1742e-01,  2.0128e-01,  2.5160e-01,  1.6774e-02],
      

## 3. Test Neurosim quantization

In [23]:
def S(bits):
    return 2. ** (bits - 1)

def Q(x, bits):
    assert bits != -1
    if bits==1:
        return torch.sign(x)
    if bits > 15:
        return x
    return torch.round(x*S(bits))/S(bits)

x = torch.randn(10).clamp(-1, 1) # 10 random numbers between -1 and 1 
print(x)
print(Q(x, 4))


tensor([-1.0000,  0.4953, -0.8715,  0.8913, -0.1896,  1.0000, -0.2816, -0.1001,
        -0.4558, -0.2664])
tensor([-1.0000,  0.5000, -0.8750,  0.8750, -0.2500,  1.0000, -0.2500, -0.1250,
        -0.5000, -0.2500])


In [12]:
g = np.arange(-2, 2.2, 0.1) # input

In [13]:
from utils import dec2bin
print(g)
dec2bin(g, 2)

[-2.00000000e+00 -1.90000000e+00 -1.80000000e+00 -1.70000000e+00
 -1.60000000e+00 -1.50000000e+00 -1.40000000e+00 -1.30000000e+00
 -1.20000000e+00 -1.10000000e+00 -1.00000000e+00 -9.00000000e-01
 -8.00000000e-01 -7.00000000e-01 -6.00000000e-01 -5.00000000e-01
 -4.00000000e-01 -3.00000000e-01 -2.00000000e-01 -1.00000000e-01
  1.77635684e-15  1.00000000e-01  2.00000000e-01  3.00000000e-01
  4.00000000e-01  5.00000000e-01  6.00000000e-01  7.00000000e-01
  8.00000000e-01  9.00000000e-01  1.00000000e+00  1.10000000e+00
  1.20000000e+00  1.30000000e+00  1.40000000e+00  1.50000000e+00
  1.60000000e+00  1.70000000e+00  1.80000000e+00  1.90000000e+00
  2.00000000e+00  2.10000000e+00]


([array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.]),
  array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1.,
         1., 1., 1., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1.])],
 [-1.0, 0.5])